In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as ply
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [15]:
df = pd.read_csv(r"C:\Users\cpilo\Downloads\df_all_teams.csv")

df_team_data = pd.get_dummies(df.team_abbr)
df_opponent_data = pd.get_dummies(df.opponent_abbr)
col_names = []
for i in df_opponent_data.columns:
    col_names.append('opp_'+i)
df_opponent_data.columns = col_names
game_city = pd.get_dummies(df.game_city)
game_time = pd.get_dummies(df.Time)
game_day = pd.get_dummies(df.Day)
vs_line = pd.get_dummies(df['vs. Line'])
qb_1 = pd.get_dummies(df['QB1'])
opp_qb_1 = pd.get_dummies(df['opp_QB1'])
skill_1 = pd.get_dummies(df['Skill1'])
opp_skill_1 = pd.get_dummies(df['opp_skill1'])
skill_2 = pd.get_dummies(df['Skill2'])
opp_skill_2 = pd.get_dummies(df['opp_skill2'])
skill_3 = pd.get_dummies(df['Skill3'])
opp_skill_3 = pd.get_dummies(df['opp_skill3'])
skill_4 = pd.get_dummies(df['Skill4'])
opp_skill_4 = pd.get_dummies(df['opp_skill4'])
skill_5 = pd.get_dummies(df['Skill5'])
opp_skill_5 = pd.get_dummies(df['opp_skill5'])
oline_1 = pd.get_dummies(df['Oline1'])
opp_oline_1 = pd.get_dummies(df['opp_oline1'])
oline_2 = pd.get_dummies(df['Oline2'])
opp_oline_2 = pd.get_dummies(df['opp_oline2'])
oline_3 = pd.get_dummies(df['Oline3'])
opp_oline_3 = pd.get_dummies(df['opp_oline3'])
oline_4 = pd.get_dummies(df['Oline4'])
opp_oline_4 = pd.get_dummies(df['opp_oline4'])
oline_5 = pd.get_dummies(df['Oline5'])
opp_oline_5 = pd.get_dummies(df['opp_oline5'])
dline_1 = pd.get_dummies(df['Dline1'])
opp_dline_1 = pd.get_dummies(df['opp_dline1'])
dline_2 = pd.get_dummies(df['Dline2'])
opp_dline_2 = pd.get_dummies(df['opp_dline2'])
dline_3 = pd.get_dummies(df['Dline3'])
opp_dline_3 = pd.get_dummies(df['opp_dline3'])
lb_1 = pd.get_dummies(df['LB1'])
opp_lb_1 = pd.get_dummies(df['opp_lb1'])
lb_2 = pd.get_dummies(df['LB2'])
opp_lb_2 = pd.get_dummies(df['opp_lb2'])
lb_3 = pd.get_dummies(df['LB3'])
opp_lb_3 = pd.get_dummies(df['opp_lb3'])
lb_4 = pd.get_dummies(df['LB4'])
opp_lb_4 = pd.get_dummies(df['opp_lb4'])
cb_1 = pd.get_dummies(df['CB1'])
opp_cb_1 = pd.get_dummies(df['opp_cb1'])
cb_2 = pd.get_dummies(df['CB2'])
opp_cb_2 = pd.get_dummies(df['opp_cb2'])
ss = pd.get_dummies(df['SS'])
opp_ss = pd.get_dummies(df['opp_ss'])
fs = pd.get_dummies(df['FS'])
opp_fs = pd.get_dummies(df['opp_fs'])

col_names = []
for i in vs_line.columns:
    col_names.append('spread_'+i)
vs_line.columns = col_names
OU_result = pd.get_dummies(df['OU Result'])
col_names = []
for i in OU_result.columns:
    col_names.append('OU_'+i)
OU_result.columns = col_names
model_vars = ['date_day', 'date_month', 'date_year', 'Week', 'Spread', 'OverUnder','days_rest', 'team_season_wins', 
              'team_season_losses', 'prev_week_total', 'prev_week_ptsdiff',
              'prev_week_ydsoff', 'prev_week_ydsdef', 'prev_week_ptsoff', 
              'prev_week_ptsdef', 'prev_week_TOoff', 'prev_week_TOdef', 
              'prev_week_seasptsdiff', 'prev_week_seasptsoff', 
              'prev_week_seasptsdef', 'prev_week_seasydsoff', 'prev_week_seasydsdef', 
              'prev_week_seasTOoff', 'prev_week_seasTOdef', 'prev_week_avgpts', 'prev_week_TOdiff',
              'prev_week_avgTOdiff', 'prev_week_avgptsdiff', 'prev_week_avgptsoff', 
              'prev_week_avgptsdef', 'prev_week_avgydsoff', 'prev_week_avgydsdef', 
              'prev_week_avgTOoff', 'prev_week_avgTOdef', 'Opp_days_rest', 'Opp_team_season_wins',
              'Opp_team_season_losses', 'opp_prev_week_total', 'opp_prev_week_pts_diff', 
              'opp_prev_week_ydsoff', 'opp_prev_week_ydsdef', 'opp_prev_week_ptsoff', 'opp_prev_week_ptsdef', 
              'opp_prev_week_TOoff', 'opp_prev_week_TOdef', 'opp_prev_week_seasptsdiff', 
              'opp_prev_week_seasptsoff', 'opp_prev_week_seasptsdef', 'opp_prev_week_seasydsoff',
              'opp_prev_week_seasydsdef', 'opp_prev_week_seasTOoff', 'opp_prev_week_seasTOdef',
              'opp_prev_week_avgpts', 'opp_prev_week_TOdiff', 'opp_prev_week_avgTOdiff',
              'opp_prev_week_avgptsdiff', 'opp_prev_week_avgptsoff', 'opp_prev_week_avgptsdef',
              'opp_prev_week_avgydsoff', 'opp_prev_week_avgydsdef', 'opp_prev_week_avgTOoff',
              'opp_prev_week_avgTOdef', 'prev_week_avgspreaddif', 'prev_week_avgoverdif', 'prev_week_avgspread',
              'prev_week_avgover', 'prev_week_seasspreadwin', 'prev_week_seasspreadloss', 'prev_week_seasspreadtie',
              'prev_week_seasoverwin', 'prev_week_seasoverloss', 'prev_week_seasovertie', 'prev_week_seastotspread',
              'prev_week_seastotover', 'prev_week_spreadwin', 'prev_week_spreadloss', 'prev_week_spreadtie',
              'prev_week_overwin', 'prev_week_overloss', 'prev_week_overtie', 'prev_week_spreaddiff', 
              'prev_week_overdiff', 'opp_prev_week_avgspreaddif', 'opp_prev_week_avgoverdif',
              'prev_week_streakw', 'prev_week_streakl', 'prev_week_streakspreadw', 'prev_week_streakspreadl',
              'prev_week_streakoverw', 'prev_week_streakoverl', 'prev_week_avgspread', 'prev_week_avgover', 
              'prev_week_seasspreadwin', 'prev_week_seasspreadloss', 'prev_week_seasspreadtie', 'prev_week_seasoverwin',
              'prev_week_seasoverloss', 'prev_week_seasovertie', 'prev_week_seastotspread', 'prev_week_seastotover',
              'prev_week_spreadwin', 'prev_week_spreadloss', 'prev_week_spreadtie', 'prev_week_overwin',
              'prev_week_overloss', 'prev_week_overtie', 'prev_week_spreaddiff', 'prev_week_overdiff',              
              'opp_prev_week_avgspread', 'opp_prev_week_avgover', 'opp_prev_week_seasspreadwin', 
              'opp_prev_week_seasspreadloss', 'opp_prev_week_seasspreadtie', 'opp_prev_week_seasoverwin',
              'opp_prev_week_seasoverloss', 'opp_prev_week_seasovertie', 'opp_prev_week_seastotspread', 
              'opp_prev_week_seastotover', 'opp_prev_week_spreadwin', 'opp_prev_week_spreadloss', 
              'opp_prev_week_spreadtie', 'opp_prev_week_overwin', 'opp_prev_week_overloss', 
              'opp_prev_week_overtie', 'opp_prev_week_spreaddiff', 'opp_prev_week_overdiff',
              'opp_prev_week_avgspread', 'opp_prev_week_avgover', 'opp_prev_week_seasspreadwin',
              'opp_prev_week_seasspreadloss', 'opp_prev_week_seasspreadtie', 'opp_prev_week_seasoverwin',
              'opp_prev_week_seasoverloss', 'opp_prev_week_seasovertie', 'opp_prev_week_seastotspread', 
              'opp_prev_week_seastotover', 'opp_prev_week_spreadwin', 'opp_prev_week_spreadloss',
              'opp_prev_week_spreadtie', 'opp_prev_week_overwin', 'opp_prev_week_overloss',
              'opp_prev_week_overtie', 'opp_prev_week_spreaddiff', 'opp_prev_week_overdiff',
              'opp_prev_week_streakw', 'opp_prev_week_streakl', 'opp_prev_week_streakspreadw',
              'opp_prev_week_streakspreadl', 'opp_prev_week_streakoverw', 'opp_prev_week_streakoverl']
              

df_rfc_data = df[model_vars]
df_final_data = pd.concat([df_rfc_data, df_team_data, df_opponent_data, game_city, qb_1, opp_qb_1,
                           skill_1, opp_skill_1, skill_2, opp_skill_2, skill_3, opp_skill_3,
                           skill_4, opp_skill_4, skill_5, opp_skill_5, oline_1, opp_oline_1,
                           oline_2, opp_oline_2, oline_3, opp_oline_3, oline_4, opp_oline_4,
                           oline_5, opp_oline_5, dline_1, opp_dline_1, dline_2, opp_dline_2,
                           dline_3, opp_dline_3, lb_1, opp_lb_1, lb_2, opp_lb_2, lb_3, opp_lb_3, 
                           lb_4, opp_lb_4, cb_1, opp_cb_1, cb_2, opp_cb_2, ss, opp_ss, fs, opp_fs,
                           game_time, game_day, vs_line, OU_result], axis = 'columns')

df_final_data = df_final_data[df_final_data['Week'] != 1]
df_final_data = df_final_data.dropna(axis = 'index')
df_train = df_final_data[df_final_data['date_year'] < 2021]
df_test = df_final_data[df_final_data['date_year'] >= 2021]
df_train = df_train.replace({True:1, False:0})
df_test = df_test.replace({True:1, False:0})
df_train

,date_day,date_month,date_year,Week,Spread,OverUnder,days_rest,team_season_wins,team_season_losses,prev_week_total,...,Sun,Thu,Tue,Wed,spread_Lost,spread_Push,spread_Won,OU_Over,OU_Push,OU_Under
1,20,9,2015,2,-2.0,46.0,7.0,1.0,0.0,50.0,...,1,0,0,0,0,0,1,1,0,0
2,27,9,2015,3,-6.5,45.0,7.0,2.0,0.0,71.0,...,1,0,0,0,0,0,1,1,0,0
3,11,10,2015,4,-3.0,45.5,14.0,3.0,0.0,54.0,...,1,0,0,0,0,0,1,1,0,0
4,18,10,2015,5,-3.5,45.0,7.0,4.0,0.0,59.0,...,1,0,0,0,1,0,0,0,0,1
5,26,10,2015,6,-8.0,49.0,8.0,4.0,1.0,38.0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,26,11,2020,11,2.5,46.0,4.0,3.0,7.0,29.0,...,0,1,0,0,0,0,1,1,0,0
3118,7,12,2020,12,6.5,43.5,11.0,4.0,7.0,57.0,...,0,0,0,0,0,0,1,0,0,1
3119,13,12,2020,13,3.0,43.5,6.0,5.0,7.0,40.0,...,1,0,0,0,0,0,1,0,0,1
3120,20,12,2020,14,6.0,44.5,7.0,6.0,7.0,38.0,...,1,0,0,0,0,0,1,0,0,1


In [16]:
df_train_y = df_train['spread_Won']
df_train_x = df_train.drop(['spread_Won', 'spread_Lost', 'spread_Push', 'OU_Over', 'OU_Push', 'OU_Under'], axis = 'columns')
df_test_y = df_test['spread_Won']
df_test_x = df_test.drop(['spread_Won', 'spread_Lost', 'spread_Push', 'OU_Over', 'OU_Push', 'OU_Under'], axis = 'columns')

In [17]:
scaler = StandardScaler()
df_train_x = scaler.fit_transform(df_train_x)
df_train_x = pd.DataFrame(df_train_x, columns = df_test_x.columns)
df_test_x = scaler.fit_transform(df_test_x)
df_test_x = pd.DataFrame(df_test_x, columns = df_train_x.columns)

In [18]:
pipe = Pipeline([('classifier' , RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     #we will try 4 different regularization penalties
     'classifier__penalty' : ['none', 'l1', 'l2', 'elasticnet'],
    'classifier__C' : np.logspace(-4, 4, 20),
     #We will try 3 different solvers
    'classifier__solver' : ['newton-cg', 'lbfgs', 'liblinear']},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
]

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(df_train_x, df_train_y)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


C:\Users\cpilo\Hello\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
600 fits failed out of a total of 1500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\cpilo\Hello\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\cpilo\Hello\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\cpilo\Hello\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

In [19]:
best_clf.best_params_

{'classifier': RandomForestClassifier(max_features=26),
 'classifier__max_features': 26,
 'classifier__n_estimators': 100}

In [20]:
best_clf.score(df_test_x, df_test_y)

0.5269360269360269

In [21]:
#The best model is a random forest classifier with 26 features
#The best performing logistic regression model was a liblinear model with l2 regularization
pd.DataFrame(best_clf.cv_results_).sort_values(by=['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__C,param_classifier__penalty,param_classifier__solver,param_classifier__max_features,param_classifier__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
289,3.386279,0.063042,0.050750,0.002141,RandomForestClassifier(max_features=26),NaN,NaN,NaN,26,100,{'classifier': RandomForestClassifier(max_feat...,0.541812,0.554007,0.515679,0.523560,0.504363,0.527884,0.017873,1
20,0.724400,0.032831,0.018039,0.004808,LogisticRegression(),0.000264,l2,liblinear,NaN,NaN,"{'classifier': LogisticRegression(), 'classifi...",0.559233,0.547038,0.501742,0.518325,0.500873,0.525442,0.023767,2
42,1.816819,0.204609,0.021874,0.003894,LogisticRegression(),0.001833,l2,newton-cg,NaN,NaN,"{'classifier': LogisticRegression(), 'classifi...",0.559233,0.543554,0.508711,0.518325,0.495637,0.525092,0.023187,3
43,0.462525,0.038485,0.021499,0.005449,LogisticRegression(),0.001833,l2,lbfgs,NaN,NaN,"{'classifier': LogisticRegression(), 'classifi...",0.559233,0.543554,0.508711,0.518325,0.495637,0.525092,0.023187,3
30,1.441522,0.103938,0.023901,0.003710,LogisticRegression(),0.000695,l2,newton-cg,NaN,NaN,"{'classifier': LogisticRegression(), 'classifi...",0.567944,0.538328,0.500000,0.516579,0.495637,0.523698,0.026726,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,0.041387,0.012660,0.000000,0.000000,LogisticRegression(),11.288379,elasticnet,newton-cg,NaN,NaN,"{'classifier': LogisticRegression(), 'classifi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,181
154,0.032573,0.009460,0.000000,0.000000,LogisticRegression(),11.288379,elasticnet,lbfgs,NaN,NaN,"{'classifier': LogisticRegression(), 'classifi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,181
208,0.042153,0.004284,0.000000,0.000000,LogisticRegression(),1438.449888,l1,lbfgs,NaN,NaN,"{'classifier': LogisticRegression(), 'classifi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,181
147,0.046475,0.009946,0.000000,0.000000,LogisticRegression(),11.288379,l1,newton-cg,NaN,NaN,"{'classifier': LogisticRegression(), 'classifi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,181


In [22]:
#we will now evaluate the performance of our best model
yhat = best_clf.best_estimator_.predict(df_test_x)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix
print("=== Confusion Matrix ===")
print(confusion_matrix(df_test_y, yhat))
print('\n')
print("=== Classification Report ===")
print(classification_report(df_test_y, yhat))

=== Confusion Matrix ===
[[183 117]
 [164 130]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.53      0.61      0.57       300
           1       0.53      0.44      0.48       294

    accuracy                           0.53       594
   macro avg       0.53      0.53      0.52       594
weighted avg       0.53      0.53      0.52       594

